In [196]:
import pandas as pd
import numpy as np 
import matplotlib as plt
import sklearn

In [197]:
df = pd.read_csv(r"D:\scrappy tuto\Twitter.csv")
df.head()

,Unnamed: 0,Id,Post Contet,Sentiment score,Post Length,Hashtag count,Content URL count,Tweet count,Followers count,Listed Count,Media Type,Published DateTime,Mentions Count,Post author verified,Likes,Shares,Comments,Impact
0,41370,6d967b125fcecba6357dbc43f8f380cf2d6d7a51,Sana all na lang.,0.0,17.0,0.0,0.0,1660.0,133.0,0,TEXT,2021-01-28 12:17:24+00:00,0.0,0.0,0.0,0.0,0.0,0.0
1,27955,22dc5f808a8589186767412f39e5c88ae9753d04,キスマイ玉森裕太「ボス恋」台本の裏話明かす\n\n@TBS_asachan @bosskoi...,19.3,84.0,0.0,1.0,318924.0,1428273.0,5454,TEXT,2021-02-24 06:51:02+00:00,0.0,1.0,10.0,1.0,0.0,110.0
2,38151,7f224e66ad757e3066d45947881bcaab91f8e57f,SuperM’s album isn’t even out yet and there’s ...,0.0,78.0,0.0,0.0,39976.0,1560.0,0,TEXT,2020-09-15 01:16:32+00:00,0.0,0.0,0.0,0.0,0.0,0.0
3,13558,5acc220e7a337efafc4c847b7f74680ca2515890,மத்திய அரசின் நலத்திட்டங்களை மக்களுக்கு சென்று...,0.0,284.0,2.0,0.0,8045.0,27191.0,26,TEXT,2021-02-14 03:53:53+00:00,0.0,1.0,460.0,219.0,4.0,6830.0
4,17071,863abf834897a06afdc459788b34abeae6298f3a,【熊本県人吉市】球磨川氾濫後のくま川鉄道の線路被害状況 hamusoku.com/archi...,-20.0,80.0,0.0,1.0,210996.0,2385.0,0,IMAGE,2020-07-04 13:06:16+00:00,0.0,0.0,0.0,0.0,0.0,0.0


from time import sleep
from textblob import TextBlob
from textblob.exceptions import NotTranslated    

def translate_comment(x):
    try:
        # Try to translate the string version of the comment
        return TextBlob(str(x)).translate(to='en')
    except NotTranslated:
        # If the output is the same as the input just return the TextBlob version of the input
        return TextBlob(str(x))

for i in range(len(df['Post Contet'])):
    # Translate one comment at a time
    df['Post Contet'].iloc[i] = translate_comment(df['Post Contet'].iloc[i])

    # Sleep for a quarter of second
    sleep(0.25)

from langdetect import detect
df['lang'] = df['Post Contet'].map(lambda x: detect(x))
df = df[df['lang']=='en']

In [198]:
df.isna().sum()
#Data set proves no null value present. Thank god handling null values takes lot of time!!!!

Unnamed: 0              0
Id                      0
Post Contet             0
Sentiment score         0
Post Length             0
Hashtag count           0
Content URL count       0
Tweet count             0
Followers count         0
Listed Count            0
Media Type              0
Published DateTime      0
Mentions Count          0
Post author verified    0
Likes                   0
Shares                  0
Comments                0
Impact                  0
dtype: int64

In [199]:
#Importing necessary libraries
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

The columns Id,Post Contet is dropped. I have not used any pre-processing operations. For unique identity unamed 0: is retained. Post content is dropped as all the necessary content is stored in columns to us from Chronicle’s engineering team. (Thanks to them )

In [200]:
df.drop(["Id",'Post Contet','Unnamed: 0',],axis=1,inplace=True)

In [201]:
df.columns

Index(['Sentiment score', 'Post Length', 'Hashtag count', 'Content URL count',
       'Tweet count', 'Followers count', 'Listed Count', 'Media Type',
       'Published DateTime', 'Mentions Count', 'Post author verified', 'Likes',
       'Shares', 'Comments', 'Impact'],
      dtype='object')

In [202]:
# This column is found to have categories!!!! Lets handle them and convert to numbering type so we fit our models easily(Bindass!!)
df['Media Type'].unique()

array(['TEXT', 'IMAGE', 'VIDEO'], dtype=object)

In [203]:
#Am doing a simple value change not using any encoding things. Keeping the approach simple. :)
df['Media Type'].replace(to_replace=['TEXT', 'IMAGE', 'VIDEO'], value=[1, 2, 3], inplace=True)

The Chronicle’s engineering team has bunched the date and time to single column. I dont like to drop that feature while modelling the models. But, to keep the model simple the column can be dropped. As of now will preserve it. 
If time doesnot permit me to use the column am dropping it. 

Below are the steps to do feature engineering for the column published datetime.
Finally am dropping Published DateTime column to avoid redundency 

In [204]:
df['Date'] = pd.to_datetime(df['Published DateTime'], format='%Y-%m-%d %H:%M:%S')

In [205]:
df['Date'].dtype

datetime64[ns, UTC]

In [206]:
df['date_delta'] = (df['Date'] - df['Date'].min())  / np.timedelta64(1,'D')

In [207]:
df['minutes'] = df['Date'].dt.hour * 60 + df['Date'].dt.minute + df['Date'].dt.second/60

In [208]:
df=df.drop(["Published DateTime","Date"],axis=1)

In [209]:
df.head()

,Sentiment score,Post Length,Hashtag count,Content URL count,Tweet count,Followers count,Listed Count,Media Type,Mentions Count,Post author verified,Likes,Shares,Comments,Impact,date_delta,minutes
0,0.0,17.0,0.0,0.0,1660.0,133.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,302.735856,737.400000
1,19.3,84.0,0.0,1.0,318924.0,1428273.0,5454,1,0.0,1.0,10.0,1.0,0.0,110.0,329.509213,411.033333
2,0.0,78.0,0.0,0.0,39976.0,1560.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,167.276921,76.533333
3,0.0,284.0,2.0,0.0,8045.0,27191.0,26,1,0.0,1.0,460.0,219.0,4.0,6830.0,319.386192,233.883333
4,-20.0,80.0,0.0,1.0,210996.0,2385.0,0,2,0.0,0.0,0.0,0.0,0.0,0.0,94.769792,786.266667


In [210]:
#Am choosing all the columns for x by dropping the impact column and considering impact column for target value 
x = df.drop("Impact",axis=1)
y = df.loc[:,["Impact"]]
x.columns
#y.columns

Index(['Sentiment score', 'Post Length', 'Hashtag count', 'Content URL count',
       'Tweet count', 'Followers count', 'Listed Count', 'Media Type',
       'Mentions Count', 'Post author verified', 'Likes', 'Shares', 'Comments',
       'date_delta', 'minutes'],
      dtype='object')

# Scatter Plot

In [212]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn import ensemble

In [216]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=13)
params = {'n_estimators': 500,
          'max_depth': 4,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'ls'}

In [217]:
estimator = ensemble.GradientBoostingRegressor(**params)
estimator.fit(x_train, y_train.values.ravel())
mse = mean_squared_error(y_test, estimator.predict(x_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

The mean squared error (MSE) on test set: 6539079.6737


In [221]:
y_pred = estimator.predict(x_test)
print("Predictions  :", y_pred)

Predictions  : [ 69266.785 110620.117   7643.394 ...    519.192    519.192    519.192]


In [222]:
print("R^2 value: ", estimator.score(x,y))

R^2 value:  0.9995129172944478
